In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
import os
import kaleido

In [2]:
botz = pd.read_csv("data/BOTZ.csv")
spy = pd.read_csv("data/SPY.csv")
icln = pd.read_csv("data/ICLN.csv")
iwda = pd.read_csv("data/IWDA.L.csv")

In [3]:
# Function to calculate the RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)

    ma_up = up.rolling(window=window).mean()
    ma_down = down.rolling(window=window).mean()

    rsi = 100 - (100 / (1 + ma_up / ma_down))
    return rsi

In [4]:
def prepare_financial_data(data):
    # Convert the 'Date' column in the DataFrame to Timestamp objects
    data['Date'] = pd.to_datetime(data['Date'])

    # Calculating 5 and 30-day SMA
    data['SMA_5'] = data['Close'].rolling(window=5).mean()
    data['SMA_30'] = data['Close'].rolling(window=30).mean()

    # Calculating 20-day SMA for Bollinger Bands
    data['SMA_20'] = data['Close'].rolling(window=20).mean()
    # Calculating the standard deviation
    data['STD_20'] = data['Close'].rolling(window=20).std()
    # Calculating Upper and Lower Bollinger Bands
    data['Upper_Band'] = data['SMA_20'] + (data['STD_20'] * 2)
    data['Lower_Band'] = data['SMA_20'] - (data['STD_20'] * 2)

    # Normalize volume bars to the range [0, 1] for better visual consistency
    data['normalized_volume'] = (data['Volume'] - data['Volume'].min()) / (data['Volume'].max() - data['Volume'].min())

    # Calculate daily returns for Volatility
    data['Daily_Return'] = data['Close'].pct_change()

    # Calculate rolling 30-day Volatility
    data['Volatility_30'] = data['Daily_Return'].rolling(window=30).std()

    # Calculate RSI
    data['RSI'] = calculate_rsi(data)

    return data

In [111]:
etfs = {}

spy = prepare_financial_data(spy)
etfs["S&P500"] = spy
botz = prepare_financial_data(botz)
etfs["AI&Robotics"] = botz
icln = prepare_financial_data(iwda)
etfs["MSCIWorld"] = iwda
iwda = prepare_financial_data(icln)
etfs["GlobalCleanEnergy"] = icln

In [58]:
def save_chart(fig, etf, start_date, end_date, chart_type):
    # Specify the file path where you want to save the PNG image
    file_path = "images/" + chart_type + etf + str(start_date) + str(end_date) + ".png"

    img = fig.to_image("png")

    # Save the PNG image to the specified file
    with open(file_path, "wb") as f:
        f.write(img)

    print(f"PNG image saved to {file_path}")

    return file_path


In [109]:
def save_label(all_data, image_name, end_date, timeframe):
    # Calculate the target date by adding the prediction offset to end_date
    target_date = pd.to_datetime(end_date) + pd.DateOffset(days=timeframe)

    # Filter 'Close' values where 'Date' equals the target_date
    price_to_predict = all_data[all_data['Date'] == target_date]['Close']

    target_date = target_date + pd.DateOffset(days=1)

    # Loop until you find a non-empty price_to_predict or until target_date exceeds the maximum date
    while target_date <= all_data['Date'].max():
        price_to_predict = all_data[all_data['Date'] == target_date]['Close']
        
        if not price_to_predict.empty:
            break
        
        target_date = target_date + pd.DateOffset(days=1)

    # If target_date exceeds the maximum date, handle the situation (e.g., raise an exception)
    if target_date > all_data['Date'].max():
        raise ValueError("No data available for prediction within the specified timeframe")

    # Calculate if the price goes up (1) or down (0) after prediction_days
    end_date = pd.to_datetime(end_date)

    # Loop until you find a non-empty price_at_end or until target_date exceeds the maximum date
    while end_date <= all_data['Date'].max():
        price_at_end = all_data[all_data['Date'] == pd.to_datetime(end_date)]['Close']
        
        if not price_at_end.empty:
            break
        
        end_date = end_date + pd.DateOffset(days=1)

    # If target_date exceeds the maximum date, handle the situation (e.g., raise an exception)
    if end_date > all_data['Date'].max():
        raise ValueError("No data available for prediction within the specified timeframe")
    
    label = 1 if price_at_end.iloc[0] > price_to_predict.iloc[0] else 0

    # Append the image name and label to the CSV file
    csv_file = 'labels.csv'
    data_to_append = pd.DataFrame({'Image': [image_name], 'Timeframe': [timeframe], 'Label': [label]})
    if os.path.exists(csv_file):
        data_to_append.to_csv(csv_file, mode='a', header=False, index=False)
    else:
        data_to_append.to_csv(csv_file, index=False)


In [11]:
def build_candlestick(etf, data, start_date, end_date):
    # Filter the data for the specified date range
    data = data[(data['Date'] >= pd.to_datetime(start_date)) & (data['Date'] <= pd.to_datetime(end_date))]

    candlestick_chart = go.Figure()

    # Add Upper Bollinger Band
    candlestick_chart.add_trace(go.Scatter(x=data['Date'], y=data['Upper_Band'], mode='lines', name='Upper Bollinger Band', line=dict(width=1, color="blue")))

    # Add Lower Bollinger Band with filled area
    candlestick_chart.add_trace(go.Scatter(x=data['Date'], y=data['Lower_Band'], mode='lines', name='Lower Bollinger Band', 
                                        fill='tonexty', fillcolor='rgba(173,216,230, 0.2)', line=dict(width=1, color="blue")))

    # Add SMA_50, SMA_200
    candlestick_chart.add_trace(go.Scatter(x=data['Date'], y=data['SMA_5'], mode='lines', name='5-day SMA'))
    candlestick_chart.add_trace(go.Scatter(x=data['Date'], y=data['SMA_30'], mode='lines', name='30-day SMA'))

    # Add Candlestick trace
    candlestick_chart.add_trace(go.Candlestick(x=data['Date'],
                                            open=data['Open'],
                                            high=data['High'],
                                            low=data['Low'],
                                            close=data['Close'],
                                            name="Candlestick"))

    # Update layout
    candlestick_chart.update_layout(
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=False
    )

    save_chart(candlestick_chart, etf, start_date, end_date, "candlestick")

In [98]:
def build_black_white_ohlc(etf, data, y1, y2, start_date, timeframe):
    # Filter the data for the specified date range
    data = data[(data['Date'] >= pd.to_datetime(start_date)) & (data['Date'] <= pd.to_datetime(start_date) + pd.DateOffset(days=timeframe))]

    # Create figure with secondary y-axis for volume
    fig = go.Figure()

    # Add 30-day Moving Average
    fig.add_trace(go.Scatter(x=data['Date'], y=data['SMA_30'], mode='lines', name='30-day SMA', line=dict(color='white')))

    # Normalize volume bars to the range [0, 1] for better visual consistency
    normalized_volume = (data['Volume'] - data['Volume'].min()) / (data['Volume'].max() - data['Volume'].min())
    # Add volume trace as bars
    fig.add_trace(go.Bar(x=data['Date'], y=normalized_volume, name='Volume', marker_color='white', yaxis='y2'))

    # Add traces for Candlestick
    fig.add_trace(go.Candlestick(x=data['Date'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'],
                                increasing_line_color='white', decreasing_line_color='white', name="Candlestick"))

    # Update layout for the y-axes ranges and chart height
    fig.update_layout(
        yaxis=dict(
            title='Price (USD)',
            range=[y1, y2],  # Set y-axis range for price
            titlefont=dict(color="white"),
            tickfont=dict(color="white"),
            showgrid=False
        ),
        yaxis2=dict(
            title='Volume',
            range=[0, 3],  # Adjusted y-axis range for volume
            overlaying='y',
            side='right',
            showgrid=False,  # Remove gridlines for secondary y-axis
            titlefont=dict(color="white"),
            tickfont=dict(color="white"),
        ),
        xaxis_rangeslider_visible=False,
        xaxis=dict(
            titlefont=dict(color="white"),
            tickfont=dict(color="white"),
        ),
        paper_bgcolor='black',
        plot_bgcolor='black',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            font=dict(color="white")
        ),
        height=600,
        width=1000
    )

    # Show the figure
    image_name = save_chart(fig, etf, start_date, timeframe, "BlackWhiteOHLC")
    return image_name

In [50]:
build_black_white_ohlc("S&P500", spy, 200, 500, "2020-01-01", "2020-12-31", 30)

PNG image saved to images/BlackWhiteOHLCS&P5002020-01-012020-12-31.png
541    376.230011
Name: Close, dtype: float64
521    373.880005
Name: Close, dtype: float64


In [465]:
build_black_white_ohlc(botz, 15, 31)

In [466]:
build_black_white_ohlc(icln, 12, 22)

In [467]:
build_black_white_ohlc(iwda, 60, 82)

In [89]:
def build_candlestick_volume(etf, data, y1, y2, start_date, end_date, timeframe):
    all_data = data
    
    # Filter the data for the specified date range
    data = data[(data['Date'] >= pd.to_datetime(start_date)) & (data['Date'] <= pd.to_datetime(end_date))]

    # Create figure with secondary y-axis for volume
    fig = go.Figure()

    # Add volume trace as bars
    fig.add_trace(go.Bar(x=data['Date'], y=data['normalized_volume'], name='Volume', marker_color='white', yaxis='y2'))

    # Add traces for Bollinger Bands, SMA, and Candlestick
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Upper_Band'], mode='lines', name='Upper Bollinger Band',
                            line=dict(width=1, color="blue")))
    fig.add_trace(go.Scatter(x=data['Date'], y=data['Lower_Band'], mode='lines', name='Lower Bollinger Band',
                            fill='tonexty', fillcolor='rgba(173,216,230, 0.2)', line=dict(width=1, color="blue")))
    #fig.add_trace(go.Scatter(x=data['Date'], y=data['SMA_5'], mode='lines', name='5-day SMA'))
    #fig.add_trace(go.Scatter(x=data['Date'], y=data['SMA_30'], mode='lines', name='30-day SMA'))
    fig.add_trace(go.Candlestick(x=data['Date'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'],
                                name="Candlestick"))

    # Update layout for the y-axes ranges and chart height
    fig.update_layout(
        yaxis=dict(
            title='Price (USD)',
            range=[y1, y2],  # Set y-axis range for price
            titlefont=dict(color="white"),
            tickfont=dict(color="white"),
            showgrid=False  
        ),
        yaxis2=dict(
            title='Volume',
            range=[0, 3],  # Adjust y-axis range for volume
            overlaying='y',
            side='right',
            showgrid=False,  # Remove gridlines for secondary y-axis
            titlefont=dict(color="white"),
            tickfont=dict(color="white"),
        ),
        xaxis=dict(
            titlefont=dict(color="white"),
            tickfont=dict(color="white"),
            showgrid=False,
            rangeslider=dict(visible=False)
        ),
        paper_bgcolor='black',
        plot_bgcolor='black',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            font=dict(color="white")
        ),
        height=600,
        width=1000
    )

    # Save image and label
    image_name = save_chart(fig, etf, start_date, end_date, "ColoredOHLC")
    save_label(all_data, image_name, end_date, timeframe)

In [57]:
build_candlestick_volume("S&P500", spy, 150, 500, "2020-01-01", "2020-12-31", 90)

PNG image saved to images/ColoredOHLCS&P5002020-01-012020-12-31.png
582    396.329987
Name: Close, dtype: float64
521    373.880005
Name: Close, dtype: float64


In [90]:
def build_ma_vol_rsi(etf, data, y1, y2, start_date, end_date, timeframe):
    all_data = data
    
    # Filter the data for the specified date range
    data = data[(data['Date'] >= pd.to_datetime(start_date)) & (data['Date'] <= pd.to_datetime(end_date))]

    # Calculate RSI and identify overbought/oversold segments
    overbought = data['RSI'] > 80
    oversold = data['RSI'] < 20

    # Create a subplot layout with two rows
    line_chart = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, 
                            row_heights=[0.9, 0.1],  # 2/3 and 1/3 height proportions
                            subplot_titles=('Price, Moving Averages, and Volatility', 'RSI'),
                            specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

    # Add closing prices, SMA_5, SMA_30 to the first row
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['Close'], mode='lines', name='Close Price', line=dict(color='white')), row=1, col=1)
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['SMA_5'], mode='lines', name='5-day SMA', line=dict(color='yellow', dash='dot')), row=1, col=1)
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['SMA_30'], mode='lines', name='30-day SMA', line=dict(color='orange', dash='dot')), row=1, col=1)

    # Add traces for Bollinger Bands, SMA, and Candlestick
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['Upper_Band'], mode='lines', name='Upper Bollinger Band',
                            line=dict(width=1, color="blue")), row=1, col=1)
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['Lower_Band'], mode='lines', name='Lower Bollinger Band',
                            fill='tonexty', fillcolor='rgba(173,216,230, 0.2)', line=dict(width=1, color="blue")), row=1, col=1)

    # Add RSI to the second row (Primary Y-Axis)
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['RSI'], mode='lines', name='14-day RSI', line=dict(width=1,color='white')), row=2, col=1)

    # Find periods of overbought and oversold to add shapes
    for i in range(1, len(data['RSI'])):
        if data['RSI'].iloc[i - 1] <= 70 and data['RSI'].iloc[i] > 70:  # Entering overbought
            start_date = data['Date'].iloc[i]
        elif data['RSI'].iloc[i - 1] > 70 and data['RSI'].iloc[i] <= 70:  # Exiting overbought
            line_chart.add_shape(type='rect', x0=start_date, x1=data['Date'].iloc[i], 
                        y0=0, y1=100, row=2, col=1, 
                        line=dict(width=0), fillcolor='red', opacity=0.2)
            line_chart.add_shape(type='rect', x0=start_date, x1=data['Date'].iloc[i], 
                        y0=y1, y1=y2, row=1, col=1, 
                        line=dict(width=0), fillcolor='red', opacity=0.2)

        if data['RSI'].iloc[i - 1] >= 30 and data['RSI'].iloc[i] < 30:  # Entering oversold
            start_date = data['Date'].iloc[i]
        elif data['RSI'].iloc[i - 1] < 30 and data['RSI'].iloc[i] >= 30:  # Exiting oversold
            line_chart.add_shape(type='rect', x0=start_date, x1=data['Date'].iloc[i], 
                        y0=0, y1=100, row=2, col=1, 
                        line=dict(width=0), fillcolor='green', opacity=0.2)
            line_chart.add_shape(type='rect', x0=start_date, x1=data['Date'].iloc[i], 
                        y0=y1, y1=y2, row=1, col=1,
                        line=dict(width=0), fillcolor='green', opacity=0.2)

    # Add Volatility to the second row (Secondary Y-Axis)
    line_chart.add_trace(go.Scatter(x=data['Date'], y=data['Volatility_30'] * 100, mode='lines', name='30-day Volatility (%)', line=dict(width=1,color='purple')), row=1, col=1, secondary_y=True)

    # Update layout
    line_chart.update_layout(
        showlegend=True,
        xaxis=dict(showgrid=False),  # Removes x-axis gridlines
        xaxis2=dict(showgrid=False),  # Removes x-axis gridlines
        yaxis=dict(showgrid=False, range=[y1, y2]),  # Set y-axis range for price and remove gridlines
        yaxis2=dict(showgrid=False),  # Removes y-axis2 gridlines for the first row
        yaxis3=dict(showgrid=False),  # Removes y-axis gridlines for the second row
        yaxis4=dict(showgrid=False),  # Removes y-axis2 gridlines for the second row
        height=600,
        width=1000,
        paper_bgcolor='black',  # Set the background color to black
        plot_bgcolor='black',  # Set the plot background color to black
        font=dict(color='white')  # Set the font color to white
    )

    # Set y-axes titles for the first row
    line_chart.update_yaxes(title_text="Price (USD)", row=1, col=1)

    # Set y-axes titles for the second row
    line_chart.update_yaxes(title_text="RSI", row=2, col=1)
    line_chart.update_yaxes(title_text="Volatility (%)", row=1, col=1, secondary_y=True)

    # Update x-axis title position to bottom right
    line_chart.update_xaxes(showticklabels=True, row=1, col=1, title="")
    line_chart.update_xaxes(showticklabels=True, row=2, col=1, title="Date")

    # Save image and label
    image_name = save_chart(line_chart, etf, start_date, end_date, "Line_RSI_Volatility")
    save_label(all_data, image_name, end_date, timeframe)

In [63]:
build_ma_vol_rsi("S&P500", spy, 200, 500, "2020-01-01", "2020-12-31", 30)

PNG image saved to images/Line_RSI_VolatilityS&P5002020-12-11 00:00:002020-12-31.png
541    376.230011
Name: Close, dtype: float64
521    373.880005
Name: Close, dtype: float64


In [478]:
build_ma_vol_rsi(botz, 18, 32)

In [479]:
build_ma_vol_rsi(icln, 12, 22)

In [480]:
build_ma_vol_rsi(iwda, 65, 82)

In [481]:
build_ma_vol_rsi(wcar, 24, 32)

In [66]:
# Checks if there is a local top detected at curr index
def rw_top(data: np.array, curr_index: int, order: int) -> bool:
    if curr_index < order * 2 + 1:
        return False

    top = True
    k = curr_index - order
    v = data[k]
    for i in range(1, order + 1):
        if data[k + i] > v or data[k - i] > v:
            top = False
            break
    
    return top

# Checks if there is a local top detected at curr index
def rw_bottom(data: np.array, curr_index: int, order: int) -> bool:
    if curr_index < order * 2 + 1:
        return False

    bottom = True
    k = curr_index - order
    v = data[k]
    for i in range(1, order + 1):
        if data[k + i] < v or data[k - i] < v:
            bottom = False
            break
    
    return bottom

def rw_extremes(data: np.array, order:int):
    # Rolling window local tops and bottoms
    tops = []
    bottoms = []
    for i in range(len(data)):
        if rw_top(data, i, order):
            # top[0] = confirmation index
            # top[1] = index of top
            # top[2] = price of top
            top = [i, i - order, data[i - order]]
            tops.append(top)
        
        if rw_bottom(data, i, order):
            # bottom[0] = confirmation index
            # bottom[1] = index of bottom
            # bottom[2] = price of bottom
            bottom = [i, i - order, data[i - order]]
            bottoms.append(bottom)
    
    return tops, bottoms

def connect_tops_and_bottoms(tops, bottoms, data):
    # Combine tops and bottoms and sort by index
    combined = sorted(tops + bottoms, key=lambda x: x[1])
    dates = [data.index[point[1]] for point in combined]
    prices = [point[2] for point in combined]

    return dates, prices

In [67]:
def identify_and_connect_doubles(tops_bottoms, time_frame, price_diff, df):
    double_patterns = []

    for i in range(len(tops_bottoms) - 1):
        for j in range(i + 1, len(tops_bottoms)):
            date_diff = df.index[tops_bottoms[j][1]] - df.index[tops_bottoms[i][1]]
            price_diff_pct = abs(tops_bottoms[j][2] - tops_bottoms[i][2]) / tops_bottoms[i][2] * 100

            # Check if the tops/bottoms are within the specified time frame and price difference
            if date_diff.days <= time_frame and price_diff_pct < price_diff:
                double_patterns.append((tops_bottoms[i], tops_bottoms[j]))

    return double_patterns


In [68]:
# Create two functions to calculate if a level is SUPPORT or a RESISTANCE level through fractal identification
def is_Suppport_Level(df, i):
  support = df['Low'][i] < df['Low'][i - 1] and df['Low'][i] < df['Low'][i + 1] and df['Low'][i + 1] < df['Low'][i + 2] and df['Low'][i - 1] < df['Low'][i - 2]
  return support

def is_Resistance_Level(df, i):
  resistance = df['High'][i] > df['High'][i - 1] and df['High'][i] > df['High'][i + 1] and df['High'][i + 1] > df['High'][i + 2] and df['High'][i - 1] > df['High'][i - 2]
  return resistance

# This function, given a price value, returns True or False depending on if it is too near to some previously discovered key level.
def distance_from_mean(level, levels, mean):
  return np.sum([abs(level - y) < mean for y in levels]) == 0

In [69]:
def add_breakout_line(fig, pattern, data, time_frame, top):
    # pattern[0] and pattern[1] are the two tops/bottoms, pattern[2] is the confirming point (trough/peak)

    if top==True:  # For double tops, use the lowest point between tops
        confirming_idx = min(range(pattern[0][1], pattern[1][1]), key=lambda x: data.iloc[x]['Close'])
    else:  # For double bottoms, use the highest point between bottoms
        confirming_idx = max(range(pattern[0][1], pattern[1][1]), key=lambda x: data.iloc[x]['Close'])

    breakout_level = data.iloc[confirming_idx]['Close']
    start_date = data.index[confirming_idx]  # Start from the second top/bottom
    end_date = start_date + pd.Timedelta(days=time_frame)

    fig.add_trace(go.Scatter(x=[start_date, end_date], y=[breakout_level, breakout_level], mode='lines', line=dict(color='white'), name='Breakout Line'))

In [92]:
def build_trading_viz(etf, data, start_date, end_date, timeframe): 
  all_data = data
  
  # Filter the data for the specified date range
  data = data[(data['Date'] >= pd.to_datetime(start_date)) & (data['Date'] <= pd.to_datetime(end_date))]

  # Initialize Plotly figure
  fig = go.Figure()

  fig.add_trace(go.Candlestick(x=data['Date'], open=data['Open'], high=data['High'], low=data['Low'], close=data['Close'],
                                name="Candlestick"))
                                
  time_frame = 45  # days
  price_diff = 2  # percentage
  data['Date'] = data['Date'].astype('datetime64[s]')
  data = data.set_index('Date')

  tops, bottoms = rw_extremes(data['Close'].to_numpy(), 10)
  data['Close'].plot()
  idx = data.index

  # Identify double tops and bottoms
  double_tops = identify_and_connect_doubles(tops, time_frame, price_diff, data)
  double_bottoms = identify_and_connect_doubles(bottoms, time_frame, price_diff, data)

  # Creating a list and feeding it the identified support and resistance levels via the Support and Resistance functions
  first_levels = []
  first_level_types = []
  for i in range(2, data.shape[0] - 2):

    if is_Suppport_Level(data, i):
      first_levels.append((i, data['Low'][i].round(2)))
      first_level_types.append('Support')

    elif is_Resistance_Level(data, i):
      first_levels.append((i, data['High'][i].round(2)))
      first_level_types.append('Resistance')

  # Clean noise in data by discarding a level if it is near another
  # (i.e. if distance to the next level is less than the average candle size for any given day - this will give a rough estimate on volatility)
  mean = np.mean(data['High'] - data['Low'])
 
  # Optimizing the analysis by adjusting the data and eliminating the noise from volatility that is causing multiple levels to show/overlapp
  levels = []
  level_types = []
  for i in range(2, data.shape[0] - 2):

    if is_Suppport_Level(data, i):
      level = data['Low'][i].round(2)

      if distance_from_mean(level, levels, mean):
        levels.append((i, level))
        level_types.append('Support')

    elif is_Resistance_Level(data, i):
      level = data['High'][i].round(2)

      if distance_from_mean(level, levels, mean):
        levels.append((i, level))
        level_types.append('Resistance')

  # Plot ETF close prices
  #fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', line=dict(color='grey', width=1),  name='Close Price'))

  # Connect tops and bottoms
  connect_dates, connect_prices = connect_tops_and_bottoms(tops, bottoms, data)

  # Add the connecting line to the figure
  fig.add_trace(go.Scatter(x=connect_dates, y=connect_prices, mode='lines', line=dict(color='white', width=2), name='Trend Line'))

  # Plot individual tops as green markers
  for top in tops:
      fig.add_trace(go.Scatter(x=[data.index[top[1]]], y=[top[2]], mode='markers', marker=dict(color='green', size=3), name='Top'))

  # Plot individual bottoms as red markers
  for bottom in bottoms:
      fig.add_trace(go.Scatter(x=[data.index[bottom[1]]], y=[bottom[2]], mode='markers', marker=dict(color='red', size=3), name='Bottom'))

  # Plot connected double tops with black lines
  for dt in double_tops:
      dt_dates = [data.index[dt[0][1]], data.index[dt[1][1]]]
      dt_prices = [dt[0][2], dt[1][2]]
      fig.add_trace(go.Scatter(x=dt_dates, y=dt_prices, mode='lines+markers', marker=dict(color='green', size=5), line=dict(color='green'), name='Double Top'))

  # Plot connected double bottoms with black lines
  for db in double_bottoms:
      db_dates = [data.index[db[0][1]], data.index[db[1][1]]]
      db_prices = [db[0][2], db[1][2]]
      fig.add_trace(go.Scatter(x=db_dates, y=db_prices, mode='lines+markers', marker=dict(color='red', size=5), line=dict(color='red'), name='Double Bottom'))

  # Add breakout lines for double tops and bottoms
  for dt in double_tops:
      add_breakout_line(fig, dt, data, 90, True)

  for db in double_bottoms:
      add_breakout_line(fig, db, data, 90, False)

  # Loop through levels and plot
  for level, level_type in zip(levels, level_types):
      index, price_level = level
      color = 'violet' if level_type == 'Support' else 'blue'
      
      # Add a horizontal line for each level
      fig.add_trace(go.Scatter(x=[data.index[index], data.index[-1]], y=[price_level, price_level], mode='lines', line=dict(color=color, width=0.5), name=level_type))

  # Update layout
  fig.update_layout(
                  xaxis=dict(
                    showgrid=False
                    ),
                  yaxis=dict(
                    title='Price',
                    #range=[200,500], 
                    showgrid=False),
                  height=600,
                  width=1000,
                  xaxis_rangeslider_visible=False,
                  paper_bgcolor='black',  # Set the background color to black
                  plot_bgcolor='black',  # Set the plot background color to black
                  font=dict(color='white'),  # Set the font color to white
                  showlegend=False)

  # Save image and label
  image_name = save_chart(fig, etf, start_date, end_date, "AlgoTrading")
  save_label(all_data, image_name, end_date, timeframe)

In [93]:
build_trading_viz("S&P500", spy, "2020-01-01", "2020-12-31", 30)

<ipython-input-92-3dd6cc7948ed>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



PNG image saved to images/AlgoTradingS&P5002020-01-012020-12-31.png
541    376.230011
Name: Close, dtype: float64
521    373.880005
Name: Close, dtype: float64


In [112]:
START = "2018-12-06"
END = "2023-12-04"
TIMEFRAMES = [7, 30, 90, 365]
PREDICTIONS = [1, 3, 7, 30, 90]
STEPS = 30
    

In [113]:
# Iterate ETFs
for key in etfs:
    print(key)
    # Iterate timeframes
    for tf in TIMEFRAMES:
        print(tf)
        start_date = pd.to_datetime(START)

        # Create images multiple times
        # While timeframe is possible to chart
        while start_date + pd.DateOffset(days=tf) < pd.to_datetime(END):
            # Create chart image
            if key == "S&P500":
                image_name = build_black_white_ohlc(key, etfs[key], 200, 500, start_date, tf)
            elif key == "MSCIWorld":
                image_name = build_black_white_ohlc(key, etfs[key], 60, 82, start_date, tf)
            elif key == "AI&Robotics":
                image_name = build_black_white_ohlc(key, etfs[key], 15, 31, start_date, tf)
            elif key == "GlobalCleanEnergy":
                image_name = build_black_white_ohlc(key, etfs[key], 12, 22, start_date, tf)
            
            # Set end date of chart
            end_date = start_date + pd.DateOffset(days=tf)

            # Iterate prediction timeframe
            for prediction in PREDICTIONS:

                # Only predict if prediction timesframe is smaller than chart timeframe
                if prediction < tf:
                    # Save label in csv file
                    save_label(etfs[key], image_name, start_date, prediction)
            
            # Set start_date
            start_date = start_date + pd.DateOffset(days=STEPS)

S&P500
7
PNG image saved to images/BlackWhiteOHLCS&P5002018-12-06 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-01-05 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-02-04 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-03-06 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-04-05 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-05-05 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-06-04 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-07-04 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-08-03 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-09-02 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-10-02 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-11-01 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-12-01 00:00:007.png
PNG image saved to images/BlackWhiteOHLCS&P5002019-12-31 00:00:007.p